# Chapter 6 — Enterprise Supply Chain Agent

**Five agentic patterns in one production pipeline:**
- Ch1 Prompt Chaining | Ch2 Routing | Ch3 Parallelization | Ch4 Reflection | Ch5 Tool Use

A global procurement system that decides whether to place a six-figure import order based on **four live data streams** — forex rates, port weather, customs holidays, and warehouse inventory.

### The Scenario

A building materials company needs to decide — right now — whether to place a bulk order for vitrified tiles arriving at the **Port of Kochi**.

Four independent data streams inform the decision:

1. **Forex Risk** — Is the Rupee weak against the Dollar?
2. **Logistics Risk** — Is a monsoon hitting Kochi?
3. **Customs Risk** — Are customs offices closed for a holiday?
4. **Inventory Need** — How many days of stock remain?

The streams are *independent* (can be fetched concurrently), but the *decision* is deeply *interdependent*. A stockout crisis doesn't override monsoon safety rules.

In [ ]:
import os
import json
import requests
import nest_asyncio
nest_asyncio.apply()

from datetime import datetime
from typing import Dict
from dotenv import load_dotenv

load_dotenv()
assert os.environ.get("GOOGLE_API_KEY"), "Set GOOGLE_API_KEY first"
print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

In [ ]:
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

---
## Tools — Three Live APIs + One Deterministic Mock

These are plain Python functions. ADK inspects the signature and docstring to generate a JSON schema for Gemini.

| Tool | Source | Purpose |
|------|--------|---------|
| `get_usd_inr_rate` | ExchangeRate-API (free, keyless) | Forex risk assessment |
| `check_port_weather` | Open-Meteo (free, keyless) | Monsoon / logistics risk |
| `check_customs_holidays` | Nager.Date (free, keyless) | Customs clearance windows |
| `fetch_erp_inventory` | Mock (deterministic) | Warehouse stock levels |

In [ ]:
def get_usd_inr_rate() -> Dict:
    """Fetch live USD/INR exchange rate from ExchangeRate-API (free, keyless).
    Returns rate, deviation from baseline, and whether Rupee is weak/strong."""
    BASELINE = 83.50
    try:
        rate = requests.get(
            "https://open.er-api.com/v6/latest/USD", timeout=5
        ).json()["rates"]["INR"]
        is_weak = rate > BASELINE
        return {
            "live_rate": round(rate, 2),
            "baseline": BASELINE,
            "deviation": f"{'+' if is_weak else ''}{((rate - BASELINE) / BASELINE * 100):.1f}%",
            "status": "WEAK_RUPEE" if is_weak else "STRONG_RUPEE",
            "impact": f"Rupee at {rate:.2f} vs baseline {BASELINE}. "
                      f"{'Expensive to import.' if is_weak else 'Favorable for imports.'}",
        }
    except Exception as e:
        return {"status": "UNKNOWN", "error": str(e)}


def check_port_weather(latitude: float = 9.93, longitude: float = 76.26) -> Dict:
    """Check 3-day rain forecast at Kochi Port via Open-Meteo (free, keyless).
    >60% rain probability = HIGH_RISK (unsafe for cargo unloading)."""
    try:
        data = requests.get(
            f"https://api.open-meteo.com/v1/forecast"
            f"?latitude={latitude}&longitude={longitude}"
            f"&daily=precipitation_probability_max&timezone=auto",
            timeout=5,
        ).json()
        probs = data["daily"]["precipitation_probability_max"][:3]
        dates = data["daily"]["time"][:3]
        max_rain = max(probs) if probs else 0
        return {
            "port": "Kochi (Cochin)",
            "forecast_days": [
                {"date": d, "rain_probability": f"{p}%"}
                for d, p in zip(dates, probs)
            ],
            "max_rain_probability": f"{max_rain}%",
            "warning": "HIGH_RISK" if max_rain > 60 else "CLEAR",
            "impact": f"Max rain {max_rain}% in 3 days. "
                      f"{'Unsafe for unloading — demurrage likely.' if max_rain > 60 else 'Port ops normal.'}",
        }
    except Exception as e:
        return {"warning": "UNKNOWN", "error": str(e)}


def check_customs_holidays(country_code: str = "IN") -> Dict:
    """Check upcoming Indian holidays via Nager.Date (free, keyless).
    Holiday within 7 days = DELAY_RISK (customs offices closed)."""
    try:
        resp = requests.get(
            f"https://date.nager.at/api/v3/NextPublicHolidays/{country_code}",
            timeout=10,
            headers={"User-Agent": "SupplyChainAgent/1.0"},
        )
        if resp.status_code != 200 or not resp.text.strip():
            return {"customs_warning": "UNKNOWN", "error": f"HTTP {resp.status_code}"}
        holidays = resp.json()
        if not holidays:
            return {"customs_warning": "CLEAR"}
        h = holidays[0]
        days = (datetime.strptime(h["date"], "%Y-%m-%d").date()
                - datetime.today().date()).days
        return {
            "next_holiday": h["name"],
            "date": h["date"],
            "days_until": days,
            "customs_warning": "DELAY_RISK" if days <= 7 else "CLEAR",
            "impact": f"{h['name']} in {days} days. "
                      f"{'Customs closed — storage fees.' if days <= 7 else 'Clearance window OK.'}",
        }
    except Exception as e:
        return {"customs_warning": "UNKNOWN", "error": str(e)}


def fetch_erp_inventory(category: str = "building-materials") -> Dict:
    """Mock ERP query — intentionally set to critical stockout.
    12 pallets / 3 per day = 4 days cover vs 21-day lead time.
    In production, replace this with your real ERP API call."""
    return {
        "sku": "Grade-A Vitrified Floor Tiles (600x600mm)",
        "current_stock": 12,
        "unit": "pallets",
        "days_of_cover": 4.0,
        "daily_consumption": 3,
        "lead_time_days": 21,
        "status": "CRITICAL",
        "impact": "12 pallets = 4 days cover vs 21-day lead time. Stockout imminent.",
    }

print("Tools defined: get_usd_inr_rate, check_port_weather, check_customs_holidays, fetch_erp_inventory")

---
## Pattern 1 — Parallelization (Chapter 3)

Four specialist agents, each with **exactly one tool and one job**. A `ParallelAgent` fires all four concurrently.

Each specialist writes findings to shared session state via `output_key` — the **blackboard pattern**.

`ParallelAgent` is **not** LLM-powered — it's a deterministic orchestrator. No tokens consumed, no hallucination possible.

In [ ]:
forex_agent = Agent(
    name="ForexAnalyst",
    model="gemini-2.5-flash",
    instruction="Call get_usd_inr_rate. Report the rate and status. "
                "Start with: STATUS: WEAK_RUPEE or STATUS: STRONG_RUPEE",
    tools=[get_usd_inr_rate],
    output_key="forex_data",
)

weather_agent = Agent(
    name="LogisticsManager",
    model="gemini-2.5-flash",
    instruction="Call check_port_weather. Report the 3-day forecast. "
                "Start with: WARNING: HIGH_RISK or WARNING: CLEAR",
    tools=[check_port_weather],
    output_key="weather_data",
)

customs_agent = Agent(
    name="CustomsBroker",
    model="gemini-2.5-flash",
    instruction="Call check_customs_holidays. Report the next holiday risk. "
                "Start with: CUSTOMS: DELAY_RISK or CUSTOMS: CLEAR",
    tools=[check_customs_holidays],
    output_key="customs_data",
)

erp_agent = Agent(
    name="InventoryManager",
    model="gemini-2.5-flash",
    instruction="Call fetch_erp_inventory. Report stock level and status. "
                "Start with: INVENTORY: CRITICAL or INVENTORY: HEALTHY",
    tools=[fetch_erp_inventory],
    output_key="erp_data",
)

data_team = ParallelAgent(
    name="ProcurementDataTeam",
    sub_agents=[forex_agent, weather_agent, customs_agent, erp_agent],
)

print("ParallelAgent ready with 4 specialist sub-agents")

---
## Pattern 2 — Prompt Chaining (Chapter 1)

The synthesizer reads all four `output_key`s from session state and reasons across them in a single context window.

It **cannot run** until all four data streams are available — the `SequentialAgent` enforces this dependency.

In [ ]:
synthesizer = Agent(
    name="StrategySynthesizer",
    model="gemini-2.5-flash",
    instruction="""You are a Supply Chain Director. Read from session state:
    forex_data, weather_data, customs_data, erp_data.

    If 'compliance_critique' exists in state, fix every issue raised.

    Recommend exactly one of:
    - [BULK ORDER]   — all conditions favorable
    - [BRIDGE ORDER] — any risk present (pallets = daily_consumption x 14)
    - [DO NOT ORDER]  — catastrophic risk only

    Include: data from each source, risk matrix, math, justification.
    Start with: RECOMMENDATION: [BULK ORDER] or [BRIDGE ORDER] or [DO NOT ORDER]""",
    output_key="draft_strategy",
)

print("StrategySynthesizer ready")

---
## Pattern 3 — Reflection (Chapter 4)

An adversarial **compliance reviewer** checks the draft against hard business rules. If violated, critique loops back to the synthesizer — up to 3 cycles.

**Hard Rules (non-negotiable):**
1. Weather HIGH_RISK (>60% rain) → BULK ORDER forbidden
2. Customs DELAY_RISK (holiday ≤7 days) → BULK ORDER forbidden
3. Forex WEAK_RUPEE (above baseline) → BULK ORDER forbidden
4. Inventory CRITICAL → DO NOT ORDER forbidden

In [ ]:
compliance_reviewer = Agent(
    name="RiskComplianceReviewer",
    model="gemini-2.5-flash",
    instruction="""You are an adversarial compliance officer.
    Read 'draft_strategy' and the raw data from state.

    HARD RULES (non-negotiable):
    Rule 1: weather HIGH_RISK (>60% rain)  -> BULK ORDER forbidden
    Rule 2: customs DELAY_RISK (<=7 days)  -> BULK ORDER forbidden
    Rule 3: forex WEAK_RUPEE (above baseline) -> BULK ORDER forbidden
    Rule 4: inventory CRITICAL             -> DO NOT ORDER forbidden

    Verify the math: bridge = daily_consumption x 14 days.

    Output MUST start with exactly one of:
    - "APPROVED: [reason]"
    - "NEEDS_REVISION: [each violation with specific data + fix instructions]" """,
    output_key="compliance_result",
)

print("RiskComplianceReviewer ready")

---
## Orchestration — The Full Pipeline

```
Phase 1: ParallelAgent fires 4 concurrent API calls
         [Forex] [Weather] [Customs] [ERP]  →  blackboard

Phase 2: Synthesize → Review → Revise loop (max 3 cycles)
         Synthesizer reads blackboard → draft
         Compliance checks draft → APPROVED or NEEDS_REVISION
         If rejected → feedback loops back to Synthesizer
```

In [ ]:
async def run_procurement_pipeline(user_message: str, max_revisions: int = 3):
    session_service = InMemorySessionService()

    # --- Phase 1: Parallel data fetch ---
    data_runner = Runner(
        agent=data_team,
        app_name="supply_chain",
        session_service=session_service,
    )
    session = await session_service.create_session(
        app_name="supply_chain", user_id="admin_1",
    )
    msg = types.Content(
        role="user",
        parts=[types.Part.from_text(text=user_message)],
    )

    print("=" * 60)
    print("PHASE 1: Parallel Data Fetch (4 concurrent API calls)")
    print("=" * 60)
    async for event in data_runner.run_async(
        user_id="admin_1", session_id=session.id, new_message=msg,
    ):
        if event.is_final_response():
            print("All four data streams collected.\n")

    # --- Phase 2: Synthesize -> Review loop ---
    critique = None
    for attempt in range(1, max_revisions + 1):
        print(f"PHASE 2 — Attempt {attempt}/{max_revisions}")
        print("-" * 40)

        # Synthesize
        synth_runner = Runner(
            agent=synthesizer,
            app_name="supply_chain",
            session_service=session_service,
        )
        prompt = "Produce the procurement strategy from state data."
        if critique:
            prompt += f" Compliance rejected your draft. Fix: {critique}"

        synth_msg = types.Content(
            role="user",
            parts=[types.Part.from_text(text=prompt)],
        )
        draft = ""
        async for event in synth_runner.run_async(
            user_id="admin_1", session_id=session.id, new_message=synth_msg,
        ):
            if event.is_final_response():
                draft = event.content.parts[0].text
                for line in draft.split("\n")[:5]:
                    if "RECOMMENDATION" in line.upper():
                        print(f"  -> {line.strip()}")

        # Compliance review
        review_runner = Runner(
            agent=compliance_reviewer,
            app_name="supply_chain",
            session_service=session_service,
        )
        review_msg = types.Content(
            role="user",
            parts=[types.Part.from_text(
                text="Review draft_strategy against the hard business rules."
            )],
        )
        review = ""
        async for event in review_runner.run_async(
            user_id="admin_1", session_id=session.id, new_message=review_msg,
        ):
            if event.is_final_response():
                review = event.content.parts[0].text

        # Check verdict
        if review.strip().upper().startswith("APPROVED"):
            print(f"\n{'=' * 60}")
            print(f"APPROVED on attempt {attempt}")
            print(f"{'=' * 60}\n")
            print(draft)
            return draft
        else:
            critique = review
            print(f"  NEEDS_REVISION — looping back to synthesizer\n")

    print(f"\nMax revisions reached. Last draft:\n{draft}")
    return draft

print("Pipeline function defined: run_procurement_pipeline()")

---
## Run It — Live Procurement Decision

This query triggers the full pipeline against **live market data**. You'll get a different forex rate and weather forecast every time you run it.

In [ ]:
result = await run_procurement_pipeline(
    "We are running low on Vitrified Tiles. "
    "Should we place a bulk order to the Kochi port today?"
)

---
## Key Takeaways

### Five Patterns Composed

| Pattern | Chapter | Role in Pipeline |
|---------|---------|------------------|
| Prompt Chaining | Ch 1 | SequentialAgent chains fetch → synthesis → review |
| Routing | Ch 2 | Enterprise router delegates procurement vs general queries |
| Parallelization | Ch 3 | ParallelAgent fires 4 independent API calls concurrently |
| Reflection | Ch 4 | Cyclic compliance loop: synthesize → review → revise |
| Tool Use | Ch 5 | 3 live APIs + 1 mock ERP with LLM-mediated reasoning |

### Architecture Lessons

- **Architecture > Model** — Same model, same tools, same data. The naive agent panics into a bulk order during a monsoon. The production pipeline places a calculated bridge order.
- **Compliance needs structure, not instructions** — Embedding rules in a prompt works until the model is under cognitive pressure from conflicting signals. A dedicated adversarial reviewer is the structural safety net.
- **Mock internals, live externals** — Deterministic ERP mock for reproducible tests. Live forex/weather APIs for real-world volatility. Best of both worlds.